In [3]:
# Import necessary libraries
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
from datetime import datetime
import openai
import json
from dotenv import load_dotenv
import os
import openai
import requests

In [4]:
# Load environment variables from .env file
load_dotenv()

# Access the OpenAI API key from environment variables
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
if not OPENAI_API_KEY:
    raise ValueError("OpenAI API key not found. Set OPENAI_API_KEY in your .env file.")
openai.api_key = OPENAI_API_KEY


## Step 1: Web Scraping to Collect External Data


In [5]:
# Define a function to collect data from a sports news website
def get_sports_data_from_web(url):
    print(f"Collecting data from {url}...")
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
    response = requests.get(url, headers=headers)
    
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Extract information such as article titles, summaries, and publication dates
        articles = []
        
        # Finding the latest news section
        news_section = soup.find('section', attrs={'data-name': 'news-feed'})
        if news_section:
            for item in news_section.find_all('div', class_='ContentList__Item'):
                title = item.find('h1') or item.find('h2') or item.find('h3')
                summary = item.find('p')
                date = item.find('time')['datetime'] if item.find('time') else datetime.now().isoformat()
                
                title_text = title.get_text(strip=True) if title else ""
                summary_text = summary.get_text(strip=True) if summary else ""
                
                if title_text:
                    articles.append({
                        'title': title_text,
                        'summary': summary_text,
                        'publication_date': date
                    })
        
        print(f"Collected {len(articles)} articles.")
        return articles
    else:
        print(f"Failed to retrieve data. Status code: {response.status_code}")
        return []

# Example URL for the Boston Celtics team page
sports_url = 'https://www.espn.com/nba/team/_/name/bos/boston-celtics'
external_articles = get_sports_data_from_web(sports_url)

# Save the articles to a CSV file for further analysis
if external_articles:
    external_articles_df = pd.DataFrame(external_articles)
    external_data_path = 'data/external/sports_articles_data.csv'
    external_articles_df.to_csv(external_data_path, index=False)
    print(f"External articles data saved to {external_data_path}.")

Collected 0 articles.


In [6]:
# Example URL (replace with a valid URL for data collection)
# sports_url = 'https://www.espn.com/nba/team/_/name/bos/boston-celtics'
# external_articles = get_sports_data_from_web(sports_url)

In [7]:
cd ..

/Users/roshni/Documents/KAGR-Project/Project/Fan_engagement


/Users/roshni/Documents/KAGR-Project/Project/Fan_engagement/.venv/lib/python3.13/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [8]:
# Save the articles to a CSV file for further analysis
external_data_path = 'data/external/sports_articles_data.csv'
external_articles_df = pd.DataFrame(external_articles)
external_articles_df.to_csv(external_data_path, index=False)
print(f"External articles data saved to {external_data_path}.")

External articles data saved to data/external/sports_articles_data.csv.


In [9]:
# Define a function to use GPT-4 to summarize articles and extract insights
def generate_summary_with_openai(article_list, max_tokens=100):
    summaries = []
    for article in article_list:
        prompt = (
            f"Summarize the following sports article in a concise manner and provide key takeaways:\n"
            f"Title: {article['title']}\n"
            f"Summary: {article['summary']}\n"
        )
        try:
            response = openai.Completion.create(
                engine="text-davinci-003",
                prompt=prompt,
                max_tokens=max_tokens,
                temperature=0.7
            )
            generated_summary = response['choices'][0]['text'].strip()
            summaries.append({
                'title': article['title'],
                'original_summary': article['summary'],
                'generated_summary': generated_summary,
                'publication_date': article['publication_date']
            })
        except Exception as e:
            print(f"Error generating summary for article '{article['title']}': {e}")
    return summaries

In [10]:
# Use OpenAI to generate summaries of the collected articles
if external_articles:
    augmented_data = generate_summary_with_openai(external_articles)
    augmented_data_path = '../data/external/augmented_sports_articles_data.csv'
    augmented_data_df = pd.DataFrame(augmented_data)
    augmented_data_df.to_csv(augmented_data_path, index=False)
    print(f"Augmented sports articles data saved to {augmented_data_path}.")
else:
    print("No articles found for summarization.")

No articles found for summarization.


In [11]:
### Step 3: Extracting Key Insights and Saving for Further Use

# Load the augmented data to analyze the insights
def analyze_augmented_data(file_path):
    df = pd.read_csv(file_path)
    # Example Analysis: Counting articles by publication date
    print("Analyzing the augmented sports articles data...")
    publication_counts = df['publication_date'].apply(lambda x: x.split('T')[0]).value_counts()
    print("Publication counts by date:")
    print(publication_counts)

# Analyze the augmented data
analyze_augmented_data(augmented_data_path)

print("RAG Data Collection Completed.")

NameError: name 'augmented_data_path' is not defined